# Any and all pacakges used for cleaning and modeling

In [1]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ziggy\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [2]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [3]:
IPO_data['CUSIP'] = IPO_data['CUSIP'].apply(lambda x: str(x).zfill(9))
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15443,15443,12/15/2023,Bayview Acquisition,000000nan
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104
15445,15445,12/19/2023,Linkage Global Inc,G5500B102


In [4]:
IPO_data = IPO_data[IPO_data['CUSIP'] != '000000nan']
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15440,15440,12/01/2023,Garden Stage Ltd,G3730L107
15441,15441,12/04/2023,Aimei Health Technology Co.,G01341117
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104


In [5]:
spac_data['CUSIP'] = spac_data['CUSIP'].apply(lambda x: str(x).zfill(9))
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1009,CF Corp.,5/25/2016,000000nan
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201
1012,Silver Run Acquisition Corp.,2/29/2016,82811P200


In [6]:
spac_data = spac_data[spac_data['CUSIP'] != '000000nan']
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1007,M III Acquisition Corp.,7/7/2016,55378T203
1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201


In [7]:
merged_IPOnSPAC = pd.merge(left=IPO_data, right=spac_data, how='left', on='CUSIP')
merged_IPOnSPAC['SPAC IPO '] = merged_IPOnSPAC['SPAC IPO '].notna().astype(int)
merged_IPOnSPAC = merged_IPOnSPAC.drop(merged_IPOnSPAC.columns[-1], axis=1)
merged_IPOnSPAC.to_csv('inputs/MergedIPO&SPAC.csv')



In [8]:
merged_IPOnSPAC

,Unnamed: 0,offer date,IPO name,CUSIP,SPAC IPO
0,0,01/30/1975,ROYSTER,078088610,0
1,1,06/09/1975,VARCO,092212610,0
2,2,06/10/1975,COORS ADOLPH,021701610,0
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,0
4,4,08/26/1975,C. F. BRAUN,010564710,0
...,...,...,...,...,...
15318,15440,12/01/2023,Garden Stage Ltd,G3730L107,0
15319,15441,12/04/2023,Aimei Health Technology Co.,G01341117,0
15320,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,0
15321,15444,12/15/2023,ZKH GROUP LTD,98877R104,0


In [9]:
# # Convert 'offer date' from 'YYYYMMDD' to 'MM/DD/YYYY'
# IPO_data['offer date'] = pd.to_datetime(IPO_data['offer date'],format='%Y%m%d')
# IPO_data['offer date'] = IPO_data['offer date'].dt.strftime('%m/%d/%Y')

# IPO_data.to_csv("inputs/IPO_data.csv")



In [10]:
# IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)

In [11]:
#Reading in ccm data

ccm_data = pd.read_csv('inputs/cleaned_ccm.csv')
ccm_data

,Unnamed: 0,datadate,cusip,scf,src,apdedate,fdate,pdate,acchg,acominc,...,seq,teq,tstk,txt,wcap,naicsh,mkvalt,Seq. No.,Code,Industry
0,20283,2014-12-31,000307108,7.0,5.0,2014-12-31,2015-04-13,2015-02-25,0.000,0.000,...,97.474,95.141,0.0,2.555,63.153,622210.0,660.8841,1821.0,622210.0,Psychiatric and Substance Abuse Hospitals
1,23764,2001-12-31,000375204,7.0,53.0,NaN,NaN,NaN,-63.000,-1699.000,...,2014.000,NaN,1750.0,105.000,479.000,335311.0,NaN,790.0,335311.0,"Power, Distribution, and Specialty Transformer..."
2,795,2014-12-31,000899104,7.0,5.0,2014-12-31,2015-04-10,2015-03-11,0.000,0.000,...,6.009,6.009,0.0,-0.552,19.890,325414.0,103.7916,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
3,5229,2001-04-30,00089C107,7.0,5.0,NaN,NaN,NaN,-1.785,1.413,...,104.664,NaN,0.0,0.037,71.958,334515.0,216.1704,770.0,334515.0,Instrument Manufacturing for Measuring and Tes...
4,5683,2000-12-31,00089J102,7.0,5.0,NaN,NaN,NaN,0.000,NaN,...,4.468,NaN,0.0,0.065,3.768,334413.0,7.9185,758.0,334413.0,Semiconductor and Related Device Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,20504,2007-12-31,Y8565N300,7.0,5.0,2007-12-31,2008-05-23,NaN,0.000,0.912,...,148.794,148.794,0.0,0.000,32.806,483111.0,550.0000,1220.0,483111.0,Deep Sea Freight Transportation
5498,14415,2004-12-31,Y8897Y230,7.0,8.0,2004-12-31,NaN,NaN,0.000,-0.248,...,321.809,NaN,0.0,0.000,98.240,483111.0,452.2538,1220.0,483111.0,Deep Sea Freight Transportation
5499,2671,2014-12-31,Y8977Y100,7.0,5.0,2014-12-31,2015-04-02,2015-02-26,0.000,0.000,...,1411.000,2451.000,0.0,20.000,143.000,213111.0,1008.2829,168.0,213111.0,Drilling Oil and Gas Wells
5500,2632,2014-12-31,Y9384M101,7.0,5.0,2014-12-31,2015-05-13,NaN,0.000,-6.000,...,283.100,835.200,0.0,16.300,46.100,424710.0,997.7975,1037.0,424710.0,Petroleum Bulk Stations and Terminals


In [12]:

merged_IPOnSPAC.rename(columns={'CUSIP' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(left=merged_IPOnSPAC, right=ccm_data, on='cusip', how='inner')

merge_df_ccm

,Unnamed: 0_x,offer date,IPO name,cusip,SPAC IPO,Unnamed: 0_y,datadate,scf,src,apdedate,...,seq,teq,tstk,txt,wcap,naicsh,mkvalt,Seq. No.,Code,Industry
0,2089,06/05/1986,Genzyme,372917104,0,110,2000-12-31,7.0,5.0,NaN,...,1750.280,NaN,0.901,57.140,438.733,325414.0,8587.5923,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
1,4619,06/16/1992,Columbia Banking System,197236102,0,6435,2000-12-31,NaN,53.0,NaN,...,78.800,NaN,0.000,6.122,NaN,522110.0,122.3405,1420.0,522110.0,Commercial Banking
2,4989,02/19/1993,Nuveen Prem Inc Muni Fund 4,6706K4105,0,4165,2000-10-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
3,5106,04/22/1993,TCW/DW Term Trust 2003,87234U108,0,4222,2001-03-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
4,5158,05/20/1993,Nuveen CT Prem Inc Muni Fund,67060D107,0,4227,2001-05-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,13368,11/15/2018,Bain Capital Specialty Finance,05684B107,0,5105,2018-12-31,7.0,5.0,2018-12-31,...,1001.629,1001.629,0.000,0.000,NaN,523910.0,863.3531,1452.0,523910.0,Miscellaneous Intermediation
2138,13376,12/07/2018,Moderna Inc,60770K107,0,5099,2018-12-31,7.0,5.0,2018-12-31,...,1530.241,1530.241,0.000,0.326,1340.203,325414.0,5020.7607,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
2139,13377,12/07/2018,Synthorx Inc,87167A103,0,5107,2018-12-31,7.0,5.0,2018-12-31,...,184.360,184.360,0.000,0.000,183.002,325414.0,545.6451,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
2140,13381,12/14/2018,360 Fin Inc,88557W101,0,5109,2018-12-31,7.0,5.0,2018-12-31,...,645.581,645.581,0.000,67.806,646.750,522220.0,NaN,1429.0,522220.0,Sales Financing


Beginning Modeling